In [1]:
import json
import os
import time

In [2]:
def get_prompts(filename):
    with open(filename) as f:
        prompts = []
        for line in f:
            prompts.append(json.loads(line))
    return prompts

In [3]:
benchmark_root = "../Quality_Analyzer/Quality_Filtered_Suggestions/"
dir_list = os.listdir(benchmark_root)

In [4]:
repair_root ="../Repair_Benchmarks_2/"

In [5]:
def get_code_up_to_line(code: str, line_number: int) -> str:
    lines = code.splitlines()
    return "\n".join(lines[:line_number])

In [ ]:
result = {}
total_prompts = 0
for file in dir_list:
    if ".DS_Store" in file:
        continue
    result[file] = {}
    prompt_list = []
    print("Processing file: ", file)
    benchmark_path = os.path.join(benchmark_root, file)
    prompts = get_prompts(benchmark_path)
    print("Number of prompts: ", len(prompts))
    dataset_type = "Java"
    if 'python' in file.lower():
        dataset_type = "Python"
    
    total_time = 0
    for prompt in prompts:
        # print("Prompt: ", prompt["task_id"])

        total_correct_suggestions = 0
        repair_prompt = None

        for i in range(len(prompt["suggestions"])):
            suggestion = prompt["suggestions"][i]
            if suggestion["Is_Vulnerable"]:
                if repair_prompt is None:
                    repair_prompt = suggestion
            if not suggestion["Is_Vulnerable"] and suggestion["Is_Compilable"]:
                 total_correct_suggestions += 1

        if total_correct_suggestions == 0 and repair_prompt is not None:
                start_time = time.time()
                new_prompt = prompt.copy()
                new_prompt.pop("suggestions", None)
                new_prompt["task_id"] = str(new_prompt["task_id"])
                new_prompt["old_prompt"] = repair_prompt["fixed_generated_text"]
                new_prompt["repair_prompt"] = repair_prompt["fixed_generated_text"]
                prompt_text = ""
                if "Security" in file:
                    total_prompt_line = len(new_prompt["Prompt"].splitlines())
                else:
                    total_prompt_line = len(new_prompt["prompt"].splitlines())

                is_first_time = False
                if dataset_type == "Python":
                    for j in range(len(repair_prompt["Analyzer_Result"])):
                        if repair_prompt["Analyzer_Result"][j]["line_number"]<=total_prompt_line:
                            continue
                        if is_first_time == False:
                            new_prompt["repair_prompt"] = get_code_up_to_line(repair_prompt["fixed_generated_text"], repair_prompt["Analyzer_Result"][j]["line_number"]-1)+"\n"
                            is_first_time = True
                        new_prompt["repair_prompt"] += "# Fix: " + repair_prompt["Analyzer_Result"][j]["issue_text"]+"\n"
                else:
                    if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]) is list:
                        for j in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"])):
                            
                            if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]) is list:
                                    for k in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"])):
                                        if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]:
                                            line_no = int(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]["@start"])
                                            if line_no<=total_prompt_line:
                                                continue
                                            if is_first_time == False:
                                                new_prompt["repair_prompt"] = get_code_up_to_line(repair_prompt["fixed_generated_text"], line_no-1)+"\n"
                                                is_first_time = True
                                            new_prompt["repair_prompt"] += "// Fix: " +  repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]+"\n"
                                            # print(prompt_text)
                                        else:
                                            pass
                                            # print(prompt_text)
                            else:
                                if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]:
                                    line_no = int(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]["@start"])
                                    if line_no<=total_prompt_line:
                                        continue
                                    if is_first_time == False:
                                        new_prompt["repair_prompt"] = get_code_up_to_line(repair_prompt["fixed_generated_text"], line_no-1)+"\n"
                                        is_first_time = True
                                    new_prompt["repair_prompt"] += "// Fix: " +  repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]+"\n"
                                    # print(prompt_text)
                                else:
                                    pass
                                    # print(prompt_text)
                    else:
                        if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]) is list:
                            for j in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"])):
                                if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]:
                                    line_no = int(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]["@start"])
                                    if line_no<=total_prompt_line:
                                        continue
                                    if is_first_time == False:
                                        new_prompt["repair_prompt"] = get_code_up_to_line(repair_prompt["fixed_generated_text"], line_no-1)+"\n"
                                        is_first_time = True
                                    new_prompt["repair_prompt"] += "// Fix: " +  repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]+"\n"
                                    # print(prompt_text)
                                else:
                                    pass

                                    # print(prompt_text)
                        else:
                            if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]:
                                line_no = int(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]["@start"])
                                if line_no<=total_prompt_line:
                                        continue
                                if is_first_time == False:
                                    new_prompt["repair_prompt"] = get_code_up_to_line(repair_prompt["fixed_generated_text"], line_no-1)+"\n"
                                    is_first_time = True
                                new_prompt["repair_prompt"] += "// Fix: " + repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]+"\n"
                                # print(prompt_text)
                            else:
                                pass
                                # print(prompt_text)
                
                # if "Security" in file:
                #     new_prompt["repair_prompt"] += prompt_text+"\n"#+new_prompt["Prompt"]
                # else:
                #     new_prompt["repair_prompt"] += prompt_text+"\n"#+new_prompt["prompt"]
                if is_first_time == False:
                    continue
                if dataset_type == "Python":
                    new_prompt["repair_prompt"] += "\n# Fixed Code: \n"
                else:
                    new_prompt["repair_prompt"] += "\n// Fixed Code: \n"
                prompt_list.append(new_prompt)

                total_time += time.time() - start_time

    print("Number of new prompts: ", len(prompt_list))
    total_prompts += len(prompt_list)
    repair_path = os.path.join(repair_root, file)
    with open(repair_path, 'w') as f:
        for prompt in prompt_list:
            f.write(json.dumps(prompt))
            f.write("\n")
    result[file]["total_time"] = total_time
print("Total prompts: ", total_prompts)

In [6]:
result = {}
total_prompts = 0
for file in dir_list:
    if ".DS_Store" in file:
        continue
    result[file] = {}
    prompt_list = []
    print("Processing file: ", file)
    benchmark_path = os.path.join(benchmark_root, file)
    prompts = get_prompts(benchmark_path)
    print("Number of prompts: ", len(prompts))
    dataset_type = "Java"
    if 'python' in file.lower():
        dataset_type = "Python"
    
    total_time = 0
    for prompt in prompts:
        # print("Prompt: ", prompt["task_id"])

        total_correct_suggestions = 0
        repair_prompt = None

        for i in range(len(prompt["suggestions"])):
            suggestion = prompt["suggestions"][i]
            if suggestion["Is_Vulnerable"]:
                if repair_prompt is None:
                    repair_prompt = suggestion
            if not suggestion["Is_Vulnerable"] and suggestion["Is_Compilable"]:
                 total_correct_suggestions += 1

        if total_correct_suggestions == 0 and repair_prompt is not None:
                start_time = time.time()
                new_prompt = prompt.copy()
                new_prompt.pop("suggestions", None)
                new_prompt["task_id"] = str(new_prompt["task_id"])
                new_prompt["old_prompt"] = repair_prompt["fixed_generated_text"]
                new_prompt["repair_prompt"] = repair_prompt["fixed_generated_text"]
                prompt_text = ""
                
                if dataset_type == "Python":
                    for j in range(len(repair_prompt["Analyzer_Result"])):
                        prompt_text += "\n# Fix: At line " + str(repair_prompt["Analyzer_Result"][j]["line_number"]) + ", " + repair_prompt["Analyzer_Result"][j]["issue_text"]
                        # print(prompt_text)
                else:
                    if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]) is list:
                        for j in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"])):
                            
                            if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]) is list:
                                    for k in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"])):
                                        if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]:
                                            prompt_text += "\n// Fix: At line " + str(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]["@start"]) + ", " + repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                            # print(prompt_text)
                                        else:
                                            prompt_text += "\n// Fix: "+ repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                            # print(prompt_text)
                            else:
                                if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]:
                                    prompt_text += "\n// Fix: At line " + str(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]["@start"]) + ", " + repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                    # print(prompt_text)
                                else:
                                    prompt_text += "\n// Fix: "+ repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                    # print(prompt_text)
                    else:
                        if type(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]) is list:
                            for j in range(len(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"])):
                                if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]:
                                    prompt_text += "\n// Fix: At line " + str(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]["@start"]) + ", " + repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                    # print(prompt_text)
                                else:
                                    prompt_text += "\n// Fix: "+ repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                    # print(prompt_text)
                        else:
                            if "@start" in repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]:
                                prompt_text += "\n// Fix: At line " + str(repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]["@start"]) + ", " + repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                # print(prompt_text)
                            else:
                                prompt_text += "\n// Fix: "+ repair_prompt["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                # print(prompt_text)
                
                if "Security" in file:
                    new_prompt["repair_prompt"] += prompt_text+"\n"#+new_prompt["Prompt"]
                else:
                    new_prompt["repair_prompt"] += prompt_text+"\n"#+new_prompt["prompt"]
                if dataset_type == "Python":
                    new_prompt["repair_prompt"] += "\n# Fixed Code: \n"
                else:
                    new_prompt["repair_prompt"] += "\n// Fixed Code: \n"
                prompt_list.append(new_prompt)
                total_time += time.time() - start_time

    print("Number of new prompts: ", len(prompt_list))
    total_prompts += len(prompt_list)
    repair_path = os.path.join(repair_root, file)
    with open(repair_path, 'w') as f:
        for prompt in prompt_list:
            f.write(json.dumps(prompt))
            f.write("\n")
    result[file]["total_time"] = total_time
print("Total prompts: ", total_prompts)

Processing file:  SecurityEval_python_codeparrot-small_128_10.jsonl
Number of prompts:  121
Number of new prompts:  23
Processing file:  SOEvalJava_PolyCoder-0.4B_128_10.jsonl
Number of prompts:  28
Number of new prompts:  1
Processing file:  CoderEval4Python_prompt_codeparrot_128_10.jsonl
Number of prompts:  230
Number of new prompts:  0
Processing file:  SOEvalPython_codegen-350M-mono_128_10.jsonl
Number of prompts:  42
Number of new prompts:  2
Processing file:  HumanEval_java_incoder-1B_128_10.jsonl
Number of prompts:  161
Number of new prompts:  15
Processing file:  SecurityEval_python_codegen-2B-multi_128_10.jsonl
Number of prompts:  121
Number of new prompts:  24
Processing file:  HumanEval_python_PolyCoder-2.7B_128_10.jsonl
Number of prompts:  164
Number of new prompts:  0
Processing file:  SOEvalJava_codegen-2B-multi_128_10.jsonl
Number of prompts:  28
Number of new prompts:  1
Processing file:  CoderEval4Python_prompt_codegen-350M-multi_128_10.jsonl
Number of prompts:  230
Nu

In [1]:
import pandas as pd

df = pd.DataFrame.from_dict(result, orient='index')
df.head()
df.to_csv("repair_time.csv")

NameError: name 'result' is not defined

In [2]:
df = pd.read_csv("repair_time.csv")
df.head()

,Unnamed: 0,Dataset,Model,total_time
0,CoderEval4Java_prompt_PolyCoder-0.4B_128_10.jsonl,CoderEval4Java_prompt,PolyCoder-0.4B,0.000042
1,CoderEval4Java_prompt_PolyCoder-160M_128_10.jsonl,CoderEval4Java_prompt,PolyCoder-160M,0.000046
2,CoderEval4Java_prompt_PolyCoder-2.7B_128_10.jsonl,CoderEval4Java_prompt,PolyCoder-2.7B,0.000039
3,CoderEval4Java_prompt_codegen-2B-multi_128_10....,CoderEval4Java_prompt,codegen-2B-multi,0.000031
4,CoderEval4Java_prompt_codegen-350M-multi_128_1...,CoderEval4Java_prompt,codegen-350M-multi,0.000032


In [3]:
def get_dataset(x):
    if 'SOEval' in x:
       return "_".join(x.split('_')[0:1])
    else:
        return "_".join(x.split('_')[0:2])

In [4]:
def get_model(x):
    if 'SOEval' in x:
        return x.split('_')[1]
    else:
        return x.split('_')[2]

In [5]:
def get_language(x):
    dataset_type = "Java"
    if "python" in x.lower():
        dataset_type = "Python"
    return dataset_type

In [8]:
df.insert(3, "Language", df["Unnamed: 0"].apply(get_language))

In [9]:
df.to_csv("repair_time.csv", index=False)

In [11]:
df = pd.read_csv("repair_time.csv")
grouped_df = df.groupby(["Model","Language"]).mean()
grouped_df.head()

total_time
Model          Language            
PolyCoder-0.4B Java        0.000040
               Python      0.000027
PolyCoder-160M Java        0.000037
               Python      0.000014
PolyCoder-2.7B Java        0.000030

In [12]:
grouped_df.to_csv("grouped_repair_time.csv")

In [13]:
grouped_df

total_time
Model              Language            
PolyCoder-0.4B     Java        0.000040
                   Python      0.000027
PolyCoder-160M     Java        0.000037
                   Python      0.000014
PolyCoder-2.7B     Java        0.000030
                   Python      0.000015
codegen-2B-mono    Python      0.000016
codegen-2B-multi   Java        0.000037
                   Python      0.000018
codegen-350M-mono  Python      0.000019
codegen-350M-multi Java        0.000038
                   Python      0.000015
codeparrot         Python      0.000022
codeparrot-small   Python      0.000098
gpt3.5             Java        0.000045
                   Python      0.000021
incoder-1B         Java        0.000023
                   Python      0.000011